In [1]:
from bitarray import bitarray

# Utilities

In [2]:
def read_file(name):
    root_path = './data/lesson01/'
    extension = '.txt'

    file = open(root_path + name + extension, encoding='utf-8-sig')
    text = file.read()
    file.close()
    
    return text

In [3]:
class BitFile(object):
    def __init__(self, header_length=4):
        '''
        Arguments:
        header_length -- length of header in bits
        '''
        self.header_length_bits = header_length * 8
        
    def encode_bit_length(self, bit_array):
        content_bits = bit_array
        content_length = len(content_bits)
        
        header_bits = bitarray(bin(content_length)[2:].zfill(self.header_length_bits))
        
        return header_bits + content_bits
    
    def decode_bit_length(self, bit_array):
        header_bits = bit_array[:self.header_length_bits]
        content_length = int(header_bits.to01(), 2)
        content_bits = bit_array[self.header_length_bits:self.header_length_bits+content_length]
        rest_bits = bit_array[self.header_length_bits+content_length:]

        return content_bits, rest_bits

    def save(self, path, bit_array):
        bits = self.encode_bit_length(bit_array)
        with open(path, 'bw') as file:
            bits.tofile(file)
            
    def load(self, path):
        with open(path, 'br') as file:
            bits = bitarray()
            bits.fromfile(file)
            
            return self.decode_bit_length(bits)[0]

# Read data

In [4]:
labels = ['czech', 'german', 'english', 'french', 'hungarian']
datasets = [read_file(label) for label in labels]

# Binary tree

In [5]:
class Node(object):
    def __init__(self, probability, char=None):
        self.probability = probability
        self.char = char
        self.code = None
        
        self.parent = None
        self.zero_child = None
        self.one_child = None
        
    @classmethod
    def deserialize(cls, succinct_code, leaf_node_chars):
        root = cls(0)
        leaf_nodes = []
        root.deserialize_traverse(root, succinct_code.copy(), leaf_node_chars.copy(), leaf_nodes)
        
        return root, leaf_nodes
            
    def deserialize_traverse(self, current, serialized, leaf_node_chars, leaf_nodes):
        left_bit = serialized.pop(0)
        if left_bit:
            child = Node(0)
            current.set_zero_child(child)
            self.deserialize_traverse(child, serialized, leaf_node_chars, leaf_nodes)

        right_bit = serialized.pop(0)
        if right_bit:
            child = Node(0)
            current.set_one_child(child)
            self.deserialize_traverse(child, serialized, leaf_node_chars, leaf_nodes)

        # handle leaf node
        if not (left_bit or right_bit):
            current.char = leaf_node_chars.pop(0)
            leaf_nodes.append(current)
    
    def serialize(self):
        '''
        Serialize tree using succinct tree encoding
        
        Sources:
         - https://en.wikipedia.org/wiki/Binary_tree#Succinct_encodings
         - https://youtu.be/3Y2weLDiUWw?t=1331
         
        Returns succinct tree encoding and list of leaf node characters in left to right order
        '''
        succinct_code = bitarray()
        leaf_node_chars = []
        self.serialize_traverse(self, succinct_code, leaf_node_chars)
        
        return succinct_code, leaf_node_chars
    
    def serialize_traverse(self, current, serialized, leaf_node_chars):
        if current.zero_child != None:
            serialized += bitarray('1')
            self.serialize_traverse(current.zero_child, serialized, leaf_node_chars)
        else:
            serialized += bitarray('0')
            
        if current.one_child != None:
            serialized += bitarray('1')
            self.serialize_traverse(current.one_child, serialized, leaf_node_chars)
        else:
            serialized += bitarray('0')
            
        # handle leaf node
        if current.zero_child == None and current.one_child == None:
            leaf_node_chars.append(current.char)            
        
    def set_zero_child(self, child):
        self.zero_child = child
        child.parent = self
        child.code = bitarray('0')
        
    def set_one_child(self, child):
        self.one_child = child
        child.parent = self
        child.code = bitarray('1')

    def decode_traverse(self, bit_array):
        if self.char != None:
            return self.char
        
        bit = bit_array.pop(0)
        if bit:
            return self.one_child.decode_traverse(bit_array)
        else:
            return self.zero_child.decode_traverse(bit_array)

    def __repr__(self):
        prefix = ''
        if self.char != None:
            prefix = f"'{self.char}' "
        return f'{prefix}{self.probability * 100:.2f}'

### Tree serialization and deseralization test

In [6]:
a = Node(0)
b = Node(0)
c = Node(0)
c.char = 'c'
d = Node(0)
d.char = 'd'
e = Node(0)
e.char = 'e'

a.set_zero_child(b)
a.set_one_child(e)
b.set_zero_child(c)
b.set_one_child(d)

serialized, node_labels = a.serialize()
Node.deserialize(serialized, node_labels)[0].serialize()

(bitarray('1100100100'), ['c', 'd', 'e'])

# Huffman coding

In [7]:
class HuffmanEncoder(object):
    def encode(self, text, encode_raw=False):
        self.compute_character_probabilities(text)
        self.parent_nodes = [Node(prob, char) for char, prob in self.character_probabilities.items()]
        self.build_tree()
        self.create_char_code_mapping()
        
        bit_array = bitarray()
        for char in text:
            bit_array += (self.char_code_mapping[char])
            
        if encode_raw:
            return bit_array
        
        bitfile = BitFile()

        print('symbols', len(self.char_code_mapping))
        
        bit_tree, labels = self.root.serialize()
        print('bit_tree size', len(bit_tree))
        bit_tree = bitfile.encode_bit_length(bit_tree)

        bit_labels = bitarray()
        bit_labels.frombytes(bytes(''.join(labels), encoding='utf8'))
        print('bit_labels size', len(bit_labels))
        bit_labels = bitfile.encode_bit_length(bit_labels)
        
        print('content size', len(bit_array))
        print(f'content bps {len(bit_array) / len(text):.2f}')

        return bit_tree + bit_labels + bit_array
        
    def build_tree(self):    
        nodes = self.parent_nodes.copy()
        while len(nodes) > 1:
            a, b = sorted(nodes, key=lambda e: e.probability)[:2]

            # merge nodes
            parent = Node(a.probability + b.probability)
            parent.set_zero_child(a)
            parent.set_one_child(b)

            nodes.remove(a)
            nodes.remove(b)
            nodes.append(parent)
            
        self.root = nodes[0]
            
    def create_char_code_mapping(self):
        mapping = {}
        for char in self.character_probabilities:
            node = [e for e in self.parent_nodes if e.char == char][0]
            char_code = bitarray()
            while node.code != None:
                char_code += node.code
                node = node.parent

            mapping[char] = char_code[::-1] # reverse, because mapping was created from leaf node

        self.char_code_mapping = mapping
        
    def compute_character_probabilities(self, text):
        probs = dict([(char, 0) for char in set(text)])

        for char in text:
            probs[char] += 1

        N = len(text)
        for char in probs:
            probs[char] /= N

        self.character_probabilities = probs

In [8]:
class HuffmanDecoder(object):
    def decode(self, bit_array):
        bit_array = bit_array.copy()
        
        bitfile = BitFile()
        tree_bits, rest = bitfile.decode_bit_length(bit_array)
        labels_bits, content = bitfile.decode_bit_length(rest)
        labels = labels_bits.tobytes().decode(encoding='utf8')
        leaf_node_chars = [char for char in labels]
        
        root, leaf_nodes = Node.deserialize(tree_bits, leaf_node_chars)
        
        # optimalization: use raw boolean array, instead of bitarray()
        # content = [c for c in content]
        # slightly better performance, but problem is probably in tree traversing
        # for short text fast enough, but computation time is not increasing linearly
        
        decoded = ''
        while len(content) > 0:
            decoded += root.decode_traverse(content)
            
        return decoded

In [9]:
class HuffmanLookupTableDecoder(object):
    def decode(self, bit_array):
        bit_array = bit_array.copy()
        
        bitfile = BitFile()
        tree_bits, rest = bitfile.decode_bit_length(bit_array)
        labels_bits, content = bitfile.decode_bit_length(rest)
        labels = labels_bits.tobytes().decode(encoding='utf8')
        leaf_node_chars = [char for char in labels]
        
        root, leaf_nodes = Node.deserialize(tree_bits, leaf_node_chars)
        self.create_char_code_mapping(leaf_nodes)
        
        decoded = ''
        content = content.to01()
        
        # optimize number of mapping lookups
        min_code_length = min([len(key) for key in self.char_code_mapping.keys()])
        
        start = 0
        i = 1
        while i <= len(content):
            x = content[start:i]
            if x in self.char_code_mapping:
                decoded += self.char_code_mapping[x]
                start = i
                i += min_code_length
            else:
                i += 1
            
        return decoded
    
    def create_char_code_mapping(self, leaf_nodes):
        mapping = {}
        for node in leaf_nodes:
            char = node.char
            char_code = bitarray()
            while node.code != None:
                char_code += node.code
                node = node.parent

            mapping[char_code[::-1].to01()] = char # reverse, because mapping was created from leaf node

        self.char_code_mapping = mapping

# Results

In [10]:
for text, label in zip(datasets, labels):
    print('Dataset:', label)
    encoder = HuffmanEncoder()
    data = encoder.encode(text)
    print('')

Dataset: czech
symbols 110
bit_tree size 438
bit_labels size 1088
content size 695565
content bps 5.10

Dataset: german
symbols 93
bit_tree size 370
bit_labels size 840
content size 1040648
content bps 4.76

Dataset: english
symbols 87
bit_tree size 346
bit_labels size 696
content size 684343
content bps 4.67

Dataset: french
symbols 98
bit_tree size 390
bit_labels size 896
content size 709381
content bps 4.95

Dataset: hungarian
symbols 107
bit_tree size 426
bit_labels size 1088
content size 942106
content bps 4.85



In [11]:
encoder = HuffmanEncoder()
data = encoder.encode(datasets[0][:10000])

decoder = HuffmanDecoder()
decoded = %time decoder.decode(data)

symbols 95
bit_tree size 378
bit_labels size 928
content size 50652
content bps 5.07
Wall time: 8.15 s


In [12]:
encoder = HuffmanEncoder()
data = %time encoder.encode(datasets[0])

decoder = HuffmanLookupTableDecoder()
decoded = %time decoder.decode(data)

symbols 110
bit_tree size 438
bit_labels size 1088
content size 695565
content bps 5.10
Wall time: 77 ms
Wall time: 190 ms


# Otázky
1. Which of these codes cannot be Huffman codes for any probability assignment and why?
- {0, 10, 11}
    - může
- {00, 01, 10, 110}
    - ne, výsledný strom by měl nelistový uzel 11, který by měl **pouze** jednoho potomka 110
- {01, 10}
    - ne, podobně jako předchozí příklad

2. Classes of codes. Consider the code {0, 01}.
- Is it prefix code?
    - ne, protože 0 je prefixem 01
- Is it uniquely decodable?
    - ano
- It it nonsingular?
    - ano, je nonsingular, protože se žádný kód neopakuje

3. Optimal word lengths.
- Can l=(1, 2, 2) be the word lengths of a binary Huffman code?
    - ano
- Can l=(2, 2, 3, 3) be the word lengths of a binary Huffman code?
    - ne